In [14]:
import pandas as pd
import numpy as np

In [15]:
link1 = "https://raw.githubusercontent.com/MuhammetRidvanInce/SHM/main/2012_arz_tablosu.csv"
link2 = "https://raw.githubusercontent.com/MuhammetRidvanInce/SHM/main/2012_yurtici_kullanim_tablosu.csv"
link3 = "https://raw.githubusercontent.com/MuhammetRidvanInce/SHM/main/2012_ithalat_kullanim_tablosu.csv"
link4 = "https://raw.githubusercontent.com/MuhammetRidvanInce/SHM/main/2012_vergi_tablosu.csv"

supplytable = pd.read_csv(link1, index_col = "index")
domusetable = pd.read_csv(link2, index_col = "index")
importusetable = pd.read_csv(link3, index_col = "index")
taxtable = pd.read_csv(link4, index_col = "index")

# Arz Tablosu

In [16]:
Vt_df = supplytable.loc["Urun_1":"Urun_64", "Sek_1": "Sek_64"]
Vt    = Vt_df.to_numpy()
V     = np.transpose(Vt)
x_df   = supplytable["Toplam Uretim"]
x      = x_df.to_numpy()[:-1]
xt     = np.transpose(x)
x_diag = np.diag(x)

# Dönüşüm Tablosu

In [17]:
D = V.dot(np.linalg.inv(x_diag))

# Yurtiçi/ İthal Kullanım ve Vergi Tablosu

In [18]:
Ud_df = domusetable.loc["Urun_1":"Urun_64", "Sek_1": "Sek_64"]
Ud = Ud_df.to_numpy()

In [19]:
Yd_df = domusetable.loc["Urun_1":"Urun_64", "Hanehalkı": "İhracat"]
Yd = Yd_df.to_numpy()

In [20]:
Um_df = importusetable.loc["Urun_1":"Urun_64", "Sek_1": "Sek_64"]
Um = Um_df.to_numpy()

In [21]:
Ym_df = importusetable.loc["Urun_1":"Urun_64", "Hanehalkı": "İhracat"]
Ym    = Ym_df.to_numpy()

In [22]:
tax_industry_df = taxtable.loc["Urun_1":"Urun_64", "Sek_1": "Sek_64"]
tax_industry    = tax_industry_df.to_numpy()

In [23]:
tax_final_df = taxtable.loc["Urun_1":"Urun_64", "Hanehalkı": "İhracat"]
tax_final    = tax_final_df.to_numpy()

In [24]:
total_tax_df = pd.concat([tax_industry_df, tax_final_df], axis = 1)
total_tax = total_tax_df.to_numpy()


# Sonuçlar

In [25]:
Bd = D.dot(Ud)
Bm = D.dot(Um)
Fd = D.dot(Yd)
Fm = D.dot(Ym)
industrytax = D.dot(tax_industry)
finaltax = D.dot(tax_final)

In [26]:
wage    = domusetable.loc["Ücret", "Sek_1": "Sek_64"].to_numpy()
capital = domusetable.loc["İşletme Artığı", "Sek_1": "Sek_64"].to_numpy()
production_tax = domusetable.loc["Üretim Üzerindeki Vergi", "Sek_1": "Sek_64"].to_numpy()
subsidies = domusetable.loc["Üretim Üzerindeki Sübvansiyon", "Sek_1": "Sek_64"].to_numpy() 

# 64/64 SHM Oluşturma

In [27]:
SAM = pd.DataFrame(0, index = domusetable.columns.to_list()[0:64] + ["EMEK", "SERMAYE", "GELİR VERGİSİ", 
                                                          "SEKTÖR ÜRÜN VERGİ", "ÜRETİM VERGİ",
                                                         "ÜRETİM SÜBVANSİYON", "HANEHALKI", "DEVLET",
                                                         "TASARRUF", "DIŞ DÜNYA (İTHALAT)", "TOPLAM"],
                  columns = domusetable.columns.to_list()[0:64] + ["EMEK", "SERMAYE", "GELİR VERGİSİ", 
                                                          "SEKTÖR ÜRÜN VERGİ", "ÜRETİM VERGİ",
                                                         "ÜRETİM SÜBVANSİYON", "HANEHALKI", "DEVLET",
                                                         "YATIRIM", "DIŞ DÜNYA (İHRACAT)", "TOPLAM"])

In [28]:
SAM.loc["Sek_1":"Sek_64","Sek_1":"Sek_64"] = Bd + industrytax 
SAM.loc["Sek_1":"Sek_64","HANEHALKI":"DIŞ DÜNYA (İHRACAT)"] = Fd + Fm + finaltax 
SAM.loc["EMEK":"SERMAYE","Sek_1":"Sek_64"] = np.array((wage, capital))
SAM.loc["SEKTÖR ÜRÜN VERGİ", "Sek_1":"Sek_64"] = np.sum(np.concatenate((industrytax, finaltax), axis = 1), axis = 1)
SAM.loc["ÜRETİM VERGİ":"ÜRETİM SÜBVANSİYON", "Sek_1":"Sek_64"] = np.array((production_tax, subsidies))
SAM.loc["DIŞ DÜNYA (İTHALAT)", "Sek_1":"Sek_64"] = np.sum(Bm, axis = 0) + np.sum(Fm, axis = 1)
SAM.loc["TOPLAM"] = np.sum(SAM)
SAM["TOPLAM"] = np.sum(SAM, axis = 1)

In [29]:
SAM.to_excel("SHM_Baslangic.xlsx")

# 5 Sektör üzerinden SHM Toplulaştırma

In [30]:
link5 = "https://raw.githubusercontent.com/MuhammetRidvanInce/SHM/main/2012_toplulastirma_tablosu.csv"
agregatevector = pd.read_csv(link5, index_col = "SIRA").iloc[:,4]
aux_SAM = pd.DataFrame(SAM.to_numpy(),
                      index   =agregatevector.to_list() + ["EMEK", "SERMAYE", "GELİR VERGİSİ", 
                                                          "SEKTÖR ÜRÜN VERGİ", "ÜRETİM VERGİ",
                                                         "ÜRETİM SÜBVANSİYON", "HANEHALKI", "DEVLET",
                                                         "TASARRUF", "DIŞ DÜNYA (İTHALAT)", "TOPLAM"],
                      columns = agregatevector.to_list() + ["EMEK", "SERMAYE", "GELİR VERGİSİ", 
                                                          "SEKTÖR ÜRÜN VERGİ", "ÜRETİM VERGİ",
                                                         "ÜRETİM SÜBVANSİYON", "HANEHALKI", "DEVLET",
                                                         "YATIRIM", "DIŞ DÜNYA (İHRACAT)", "TOPLAM"])

In [31]:
SAM2 = pd.DataFrame(0, index = ["TARIM", "TİCARET VE HİZMET", "ULAŞIM", "İNŞAAT", "SANAYİ", 
                            "EMEK", "SERMAYE", "GELİR VERGİSİ", "SEKTÖR ÜRÜN VERGİ", "ÜRETİM VERGİ",
                            "ÜRETİM SÜBVANSİYON", "HANEHALKI", "DEVLET",
                            "TASARRUF", "DIŞ DÜNYA (İTHALAT)", "TOPLAM"],
                    
                   columns = ["TARIM", "TİCARET VE HİZMET", "ULAŞIM", "İNŞAAT", "SANAYİ",
                             "EMEK", "SERMAYE", "GELİR VERGİSİ", "SEKTÖR ÜRÜN VERGİ", "ÜRETİM VERGİ",
                             "ÜRETİM SÜBVANSİYON", "HANEHALKI",  "DEVLET",
                             "YATIRIM", "DIŞ DÜNYA (İHRACAT)", "TOPLAM"])

In [32]:
for row in aux_SAM.index:
    for col in aux_SAM.columns:
         SAM2.loc[row, col] = np.sum(np.sum(aux_SAM.loc[row, col]))

In [33]:
SAM2.to_excel("SHM_baslangic_Toplulastirilmis.xlsx")